<a href="https://colab.research.google.com/github/meetmehedi/datas/blob/main/Food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow


In [ ]:
# train_model.py
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import numpy as np

os.makedirs("models", exist_ok=True)
BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS = 6   # increase later for better accuracy

def preprocess(example):
    image = tf.image.resize(example['image'], (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = example['label']
    return image, label

def prepare_datasets():
    print("Downloading Food-101 (this may take a while)...")
    ds_train, ds_info = tfds.load('food101', split='train', with_info=True, shuffle_files=True)
    ds_test = tfds.load('food101', split='validation')  # TFDS uses validation as test for food101
    num_classes = ds_info.features['label'].num_classes
    print(f"Found {num_classes} classes.")

    train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train = train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    val = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    val = val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return train, val, num_classes, ds_info

def build_model(num_classes):
    base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                             include_top=False,
                                             weights='imagenet')
    base.trainable = False  # freeze base for quick MVP

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def main():
    train_ds, val_ds, n_classes, ds_info = prepare_datasets()
    model = build_model(n_classes)
    print(model.summary())

    # quick training
    model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

    model_path = "models/food101_mobilenetv2.h5"
    print(f"Saving model to {model_path}")
    model.save(model_path)

    # Save label names
    label_names = ds_info.features['label'].names
    import json
    with open("models/label_names.json", "w") as f:
        json.dump(label_names, f)
    print("Saved label names to models/label_names.json")
    print("Done.")

if __name__ == "__main__":
    main()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# app.py
import streamlit as st
import numpy as np
from PIL import Image
import os
import json
import requests
import io

MODEL_PATH = "models/food101_mobilenetv2.h5"
LABELS_PATH = "models/label_names.json"
IMG_SIZE = 224

st.set_page_config(page_title="FoodSave MVP", layout="centered")

st.title("FoodSave — MVP")
st.write("Upload a food photo. The app will try to identify the food and suggest actions to reduce waste.")

# Load model
model = None
labels = None
if os.path.exists(MODEL_PATH) and os.path.exists(LABELS_PATH):
    import tensorflow as tf
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(LABELS_PATH, "r") as f:
        labels = json.load(f)
else:
    st.warning("Model not found. Run `python train_model.py` first to create the model.")
    st.info("You can still upload images to see local preview and suggestions.")

uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded image", use_column_width=True)
    st.write("")

    if model is not None and labels is not None:
        # preprocess
        img = image.resize((IMG_SIZE, IMG_SIZE))
        x = np.array(img) / 255.0
        x = np.expand_dims(x, axis=0)

        preds = model.predict(x)[0]
        top_idx = np.argsort(preds)[::-1][:3]
        st.markdown("### Predictions (top 3):")
        for i in top_idx:
            st.write(f"- **{labels[i]}** — confidence: {preds[i]:.3f}")

        predicted = labels[top_idx[0]]

        st.markdown("### Open Food Facts lookup (by predicted label)")
        try:
            # simple search by predicted label
            query = predicted.replace(" ", "+")
            url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={query}&search_simple=1&action=process&json=1&page_size=5"
            r = requests.get(url, timeout=10)
            data = r.json()
            if data.get("products"):
                st.write(f"Found {len(data['products'])} product(s) related to *{predicted}* (sample):")
                for p in data["products"][:3]:
                    name = p.get("product_name") or p.get("generic_name") or "Unnamed product"
                    brands = p.get("brands", "")
                    nutri = p.get("nutriments", {})
                    st.write(f"- **{name}** — {brands}")
                    if 'expiration_date' in p:
                        st.write(f"  - expiry: {p['expiration_date']}")
                    # show minimal nutrition if exists
                    if nutri:
                        energy = nutri.get("energy-kcal_100g") or nutri.get("energy_100g")
                        if energy:
                            st.write(f"  - energy (per 100g): {energy}")
            else:
                st.write("No matching product found on Open Food Facts for this predicted label.")
        except Exception as e:
            st.write("Open Food Facts lookup failed:", e)

        st.markdown("### Suggestions to reduce waste")
        st.write("- If expiry is near: cook soon or freeze.")
        st.write("- Convert to recipes that use leftover ingredients (soups, stir-fry).")
        st.write("- Share via community apps or donor platforms.")
    else:
        st.info("Model not available. While the model is missing you can still use the app UI.")
        st.markdown("### Suggestions (based on generic rules)")
        st.write("- Smell & visual check for spoilage.")
        st.write("- If fruit/veg: use within 2–3 days or pickle/preserve.")
        st.write("- If packaged: check packaging & expiry date.")

st.sidebar.header("MVP Tips")
st.sidebar.write("1. Run `python train_model.py` to create the model (uses Food-101).")
st.sidebar.write("2. For better accuracy: increase epochs and unfreeze the base model.")
st.sidebar.write("3. Collect local photos using the app (consent) to fine-tune model for your region.")

In [ ]:
%pip install streamlit